## Image Processing Using Python

In [ ]:
from PIL import Image

In [ ]:
pic = Image.open('white-flower.jpg')

In [ ]:
pic

In [ ]:
type(pic)

In [ ]:
import numpy as np

In [ ]:
pic_arr = np.asarray(pic)

In [ ]:
pic_arr

In [ ]:
pic_arr.shape

In [ ]:
pic_arr / 255

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(pic_arr)
plt.axis(False);

In [ ]:
pic_red = pic_arr.copy()

In [ ]:
pic_red

In [ ]:
# RGB

pic_red[:, :, 1] = 0 # GREEN Channel : set -> 0
pic_red[:, :, 2] = 0 # BLUE Channel : Set -> 0

In [ ]:
pic_red

In [ ]:
# RGB

pic_green = pic_arr.copy()
pic_green[:, :, 0] = 0 # Red channel is zeroed -> 0
pic_green[:, :, 2] = 0 # BLUE channel is zeroed -> 0

pic_green

In [ ]:
pic_blue = pic_arr.copy()

pic_blue[:, :, 0] = 0
pic_blue[:, :, 1] = 0
pic_blue

In [ ]:
import cv2

In [ ]:
img = cv2.imread("/content/white-flower.jpg")

In [ ]:
img

In [ ]:
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
img_rgb

In [ ]:
img_gray = cv2.imread("/content/white-flower.jpg", cv2.IMREAD_GRAYSCALE)

In [ ]:
plt.imshow(img_gray, cmap="gray")

In [ ]:
img_resize = cv2.resize(img, (200, 400))
plt.imshow(img_resize)

In [ ]:
w_ratio = 0.5
h_ratio = 0.5

new_img = cv2.resize(img, (0, 0), fx=w_ratio, fy=h_ratio)

In [ ]:
new_img

## Generating Faces using Diffusion Models

In [ ]:
from diffusers import DDPMPipeline # Denoising Diffusion Probabilistic Model

In [ ]:
ddpm = DDPMPipeline.from_pretrained("google/ddpm-celebahq-256").to("cuda")

In [ ]:
image = ddpm(num_inference_steps=30).images[0]

In [ ]:
image

In [ ]:
from diffusers import DDPMScheduler, UNet2DModel

In [ ]:
scheduler = DDPMScheduler.from_pretrained("google/ddpm-celebahq-256")

In [ ]:
scheduler

In [ ]:
model = UNet2DModel.from_pretrained("google/ddpm-celebahq-256").to("cuda")

In [ ]:
model

In [ ]:
scheduler.set_timesteps(50)

In [ ]:
scheduler.timesteps

In [ ]:
import torch

In [ ]:
sample_size = model.config.sample_size

In [ ]:
sample_size

In [ ]:
noise = torch.randn((1, 3, sample_size, sample_size), device="cuda")

In [ ]:
noise

In [ ]:
input = noise
plt.imshow(input.squeeze().permute(1, 2, 0).detach().to("cpu").numpy());

In [ ]:
plt.hist(input.squeeze().permute(1, 2, 0).detach().to("cpu").numpy().flatten());

In [ ]:
scheduler.timesteps, len(scheduler.timesteps)

In [ ]:
for t in scheduler.timesteps:
    with torch.inference_mode(): # enable and turn on inference mode
        noisy_residual = model(input, t).sample

        previous_noisy_sample = scheduler.step(noisy_residual, t, input).prev_sample

        input = previous_noisy_sample

In [ ]:
from PIL import Image
import numpy as np

In [ ]:
image = (input / 2 + 0.5).clamp(0, 1).squeeze()

In [ ]:
image = (image.permute(1, 2, 0) * 255).round().to(torch.uint8).cpu().numpy()

In [ ]:
image = Image.fromarray(image)

In [ ]:
image

## Generating Images using a prompt with Diffusion Models

In [ ]:
import torch
from diffusers import DiffusionPipeline

# switch to "mps" for apple devices
pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", dtype=torch.bfloat16, device_map="cuda")

prompt = "Astronaut in a jungle, cold color palette, muted colors, detailed, 8k"
image = pipe(prompt).images[0]

In [ ]:
image

In [ ]:
prompt = "a cute and adorable bunny, with huge clear eyes, holding a bunch of flowers, in the style of cute pixar character"
image = pipe(prompt).images[0]
image

In [ ]:
prompt = "a cute and lovely 4-year-old baby girl with a magical power, cosmic energy, lovely smile, digital painting, hyperrealistic, highly detailed, sharp, masterpiece"
image = pipe(prompt).images[0]
image

In [ ]:
prompt = "A giraffe with top hat" # [A] [Giraffe] [with] => [0.5, 0.3, 0.2]
image = pipe(prompt).images[0]
image